In [25]:
import pygraphviz as pgv
import pandas as pd
import dowhy
from dowhy import CausalModel
import warnings
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sys import path
import os
for dirname, _, filenames in os.walk('../resources/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
path.append(os.path.realpath('../'))

from custom import funtions

warnings.filterwarnings('ignore')

../resources/allColProSol.xlsx
../resources/dataset_a_2021.csv
../resources/dataset_a_2021c2.csv
../resources/dataset_a_2021v1.xlsx
../resources/dataset_unab_ORIGINAL SIN P1.csv
../resources/dataset_unab_P1.csv
../resources/exitoFallidoEnviosProgramaSol.xlsx
../resources/exitoFallidoProgramaSol.xlsx
../resources/exitosoFallidosEnviosAllColProSol.xlsx
../resources/exitosoFallidosEnviosAllColSol.xlsx
../resources/exitososFallidosEnviosSol.xlsx
../resources/Hito12sinColCeroExitosoFallidosSol.xlsx
../resources/sinColCeroExitosoFallidosEnviosSol.xlsx
../resources/sinColCeroExitosoFallidosSol.xlsx
../resources/v2_hitosExitoFalloColESol1.csv
../resources/v2_hitosExitoFalloColESol1Prograna.csv
../resources/v2_hitosExitoFalloSol1Programa.csv
../resources/causalidad\causalidad.dot
../resources/causalidad\causalidad.png
../resources/causalidad\causalidad2.dot
../resources/causalidad\causalidad_hito1.dot
../resources/causalidad\graph.png
../resources/causalidad\graph_hito1.png


In [26]:
# Read and preview data
df = pd.read_csv("../resources/v2_hitosExitoFalloColESol1.csv", delimiter=";", skipinitialspace=True)

In [27]:
# Convertir la columna "sol1" a números de punto flotante
df['sol1'] = df['sol1'].astype(float)
df['exitosos'] = df['exitosos'].astype(int)
df['fallidos'] = df['fallidos'].astype(int)

In [28]:
#creando columna aprobado y con la funcion set_in_aprobado_nota poblamos la nueva columna.
df['aprobado']=df.apply(lambda x: funtions.set_in_aprobado_nota(x['sol1']),axis = 1 )
# #reliminando columna sol1, ya que usaremos aprobado
df = df.drop('sol1', axis=1)
#revisamos la existencia de la nueva columna.
print(df.columns)

Index(['hito1', 'hito2', 'exitosos', 'fallidos', 'e0', 'e1', 'e2', 'e3', 'e4',
       'e5', 'e6', 'e7', 'e8', 'e9', 'e10', 'e11', 'e12', 'e13', 'e14', 'e15',
       'e16', 'e17', 'e18', 'e19', 'e20', 'e21', 'e22', 'e23', 'e24', 'e25',
       'e26', 'e27', 'e28', 'e29', 'e30', 'e31', 'e32', 'e33', 'e34', 'e35',
       'e36', 'e37', 'e38', 'e39', 'e40', 'e41', 'e42', 'e43', 'e44', 'e45',
       'e46', 'e47', 'e48', 'e49', 'e50', 'e51', 'e52', 'aprobado'],
      dtype='object')


In [29]:
# Crear un objeto del tipo AGraph
graph = pgv.AGraph(directed=True)

In [30]:
# Agregar nodos al gráfico
for column in df.columns:
    graph.add_node(column)

# Agregar aristas al gráfico
for column in df.columns:
    if column != 'exitosos' and column != 'aprobado':
        graph.add_edge(column, 'exitosos')
        graph.add_edge(column, 'aprobado')

# Guardar el grafo en un archivo DOT en la ruta deseada
graph.write("../resources/causalidad/causalidad_exitosos.dot")
# Visualizar el gráfico
graph.layout(prog='dot')
graph.draw('../resources/causalidad/graph_exitosos.png')

In [31]:
# Lee el archivo DOT
graph_path = "../resources/causalidad/causalidad_exitosos.dot"
model = CausalModel(
    data=df,
    graph=graph_path,
    treatment='exitosos',
    outcome='aprobado'
)

In [32]:
# Identificar el estimando causal
identified_estimand = model.identify_effect()

# Estimar el efecto causal utilizando propensity score matching
estimate = model.estimate_effect(
    identified_estimand, method_name="backdoor.propensity_score_matching"
)

# Imprimir el resultado
print(estimate)


propensity_score_matching
*** Causal Estimate ***

## Identified estimand
No directed path from ['exitosos'] to ['aprobado'] in the causal graph.
Causal effect is zero.
## Realized estimand
None
## Estimate
Mean value: 0

